--- Day 7: Some Assembly Required ---

This year, Santa brought little Bobby Tables a set of wires and bitwise logic gates! Unfortunately, little Bobby is a little under the recommended age range, and he needs help assembling the circuit.

Each wire has an identifier (some lowercase letters) and can carry a 16-bit signal (a number from 0 to 65535). A signal is provided to each wire by a gate, another wire, or some specific value. Each wire can only get a signal from one source, but can provide its signal to multiple destinations. A gate provides no signal until all of its inputs have a signal.

The included instructions booklet describes how to connect the parts together: x AND y -> z means to connect wires x and y to an AND gate, and then connect its output to wire z.

For example:

123 -> x means that the signal 123 is provided to wire x.

x AND y -> z means that the bitwise AND of wire x and wire y is provided to wire z.

p LSHIFT 2 -> q means that the value from wire p is left-shifted by 2 and then provided to wire q.

NOT e -> f means that the bitwise complement of the value from wire e is provided to wire f.

Other possible gates include OR (bitwise OR) and RSHIFT (right-shift). If, for some reason, you'd like to emulate the circuit instead, almost all programming languages (for example, C, JavaScript, or Python) provide operators for these gates.

For example, here is a simple circuit:

123 -> x
456 -> y
x AND y -> d
x OR y -> e
x LSHIFT 2 -> f
y RSHIFT 2 -> g
NOT x -> h
NOT y -> i
After it is run, these are the signals on the wires:

d: 72
e: 507
f: 492
g: 114
h: 65412
i: 65079
x: 123
y: 456
In little Bobby's kit's instructions booklet (provided as your puzzle input), what signal is ultimately provided to wire a?


In [1]:
#Q7.
import numpy as np
logic = []
for i in range(0,16):
    logic.append(2**i)
logic =sorted(logic, reverse=True)
print(logic)


[32768, 16384, 8192, 4096, 2048, 1024, 512, 256, 128, 64, 32, 16, 8, 4, 2, 1]


In [2]:
def bitwise(number):
    if type(number)==int:
        size=16
        number_bit=np.zeros((size),dtype=np.int16)
        i=0
        for item in logic:
            if number >= item:
                number -=item      
                number_bit[i]=1
                i +=1
            else:
                i+=1
        return((number_bit))
x=bitwise(123)
y=bitwise(456)

###############

def bit16():
    size=16
    num=np.zeros((size),dtype=np.int16)
    return num
def one():
    num=bit16()
    num[15]=1
    return num


In [3]:
def num_and (num1, num2):
    size = 16
    new_num = np.zeros((size),dtype=np.int16)
    for i in range(0,size):
        if (num1[i]==1) and (num2[i]==1):
            new_num[i]=1
    return new_num

def num_or (num1, num2):
    size=16
    new_num=np.zeros((size),dtype=np.int16)
    for i in range(0,size):
        if num1[i]==1 or num2[i]==1:
            new_num[i]=1
    return new_num 

def num_not (num):
    size=16
    new_num=np.zeros((size),dtype=np.int16)
    for i in range(0,size):
        if num[i]==0:
            new_num[i]=1
    return new_num 


def r_shift(num,shift):
    size=16
    new_num=np.zeros((size),dtype=np.int16)
    for i in range(0,size-shift):
        new_num[i+shift]=num[i]
    return new_num

def l_shift(num, shift):
    size=16
    new_num=np.zeros((size),dtype=np.int16)
    for i in range(shift,size):
        new_num[i-shift]=num[i]
    return new_num   

In [4]:
with open(r"C:\Users\myavu\OneDrive\Desktop\Questions\2015\7.txt") as f:
    lines =f. read().splitlines()
    
###############

def parsing(line):
    command_table= ["0", "0", "0", "0"]
    line= line.split(" ")
    if len(line)== 3:
        command_table[0]="bitwise"
        command_table[1]=line[0]
        command_table[2]=line[2]            
    elif len(line)==4:
        command_table[0]=line[0]
        command_table[1]=line[1]
        command_table[2]=line[3]
    elif len(line)==5:
        command_table[0]=line[1]
        command_table[1]=line[0]
        command_table[2]=line[2]
        command_table[3]=line[4]
    return command_table 

#for line in lines:
    #command=parsing (line)
    #print(command)


In [5]:
def find_firstnumbers(x):
    my_dict={}
    for line in x:        
        command=parsing(line)
        if command[0]=="bitwise":
            try:         
                my_dict[command[2]]=bitwise(int(command[1]))
            except:                              
                my_dict[command[1]]=bit16()
                my_dict[command[2]]=bit16()
        elif command[0]=="AND":
            if command[1]=='1':
                my_dict[command[1]]=one()
                my_dict[command[3]]=bit16()
            else:           
                my_dict[command[3]]=bit16()
        elif command[0]=="OR":
            my_dict[command[3]]=bit16()
        elif command[0]=="NOT":
            my_dict[command[2]]=bit16()
        elif command[0]=="LSHIFT":
            my_dict[command[3]]=bit16()
        elif command[0]=="RSHIFT":
            my_dict[command[3]]=bit16()

    return my_dict

#####################
#find_firstnumbers(lines)


In [6]:
lines_count=0
for line in lines:
    lines_count+=1
    command=parsing (line)
print(lines_count)


339


In [7]:
def find_numbers(lines):
    new_dict=find_firstnumbers(lines)
    for i in range(339):
        for line in lines:
            command=parsing(line)
            if command[0]=="AND":
                new_dict[command[3]]=num_and(new_dict[command[1]], new_dict[command[2]])
            elif command[0]=="OR":
                new_dict[command[3]]=num_or(new_dict[command[1]], new_dict[command[2]])
            elif command[0]=="LSHIFT":
                new_dict[command[3]]=l_shift(new_dict[command[1]], int(command[2]))
            elif command[0]=="RSHIFT":
                new_dict[command[3]]=r_shift(new_dict[command[1]], int(command[2]))
            elif command[0]=="NOT":
                new_dict[command[2]]=num_not(new_dict[command[1]])
    return new_dict

numbers=find_numbers(lines)


In [8]:
def converter(numbers):
    values=numbers.copy()
    for key, value in dict.items(numbers):
        val=0
        number =np.flip(value) 
        for i in range(0,16):
            val += (2**i) * number[i]
        values[key]=val    
    return values


converter(numbers)

#Note that lx=a
print(converter(numbers)['lx'])


16076


--- Part Two ---

Now, take the signal you got on wire a, override wire b to that signal, and reset the other wires (including wire a). What new signal is ultimately provided to wire a?

In [9]:
with open(r"C:\Users\myavu\OneDrive\Desktop\Questions\2015\7b.txt") as f:
    lines =f. read().splitlines()
    
def new_count(x):    
    numb=find_numbers(x)
    new_result= converter(numb)
    return new_result
    
    
#Note that lx=a
print(new_count(lines)['lx'])

2797
